In [286]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

In [287]:
# подключаемся к файлу
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/analog-oven-310306-0db0982e2069.json', scope)
client = gspread.authorize(creds)

In [288]:
sh = client.open_by_key('1B_7BIGEQVRcdE9RY9U5fhNMCPzIZbTbSnXWLLJeKU2M')

In [289]:
#загружаем листы
legend = sh.worksheet('Легенда')
transactions = sh.worksheet('transactions')
clients = sh.worksheet('clients')
managers = sh.worksheet('managers')
leads = sh.worksheet('leads')

In [290]:
#создаем датафреймы
tdf = pd.DataFrame(transactions.get_all_records())
cdf = pd.DataFrame(clients.get_all_records())
mdf = pd.DataFrame(managers.get_all_records())
ldf = pd.DataFrame(leads.get_all_records())

In [291]:
ldf.loc[ldf['d_utm_source']=='vkontakte', 'd_utm_source'] = 'vk'
ldf.loc[ldf['d_utm_source']=='ycard#!/tproduct/225696739-1498486363994', 'd_utm_source'] = 'ycard'
ldf.loc[ldf['d_utm_source']=='ig', 'd_utm_source'] = 'instagram'
ldf.loc[ldf['d_utm_source']=='insta', 'd_utm_source'] = 'instagram'


In [313]:
report = pd.DataFrame()
#количество заявок
report['leads'] = ldf.merge(mdf, how='left', right_on='manager_id', left_on='l_manager_id').groupby(['d_utm_source', 'd_club', 'd_manager'])['l_client_id'].count()


In [314]:
#количество мусорных заявок (на основании заявки не создан клиент) 
report['trash_leads'] = ldf[ldf.merge(cdf, how='left', right_on='client_id', left_on='l_client_id')['client_id'].isna()].merge(mdf, how='left', right_on='manager_id', left_on='l_manager_id').groupby(['d_utm_source', 'd_club', 'd_manager'])['l_client_id'].count()

In [315]:
#ищем немусорные заявки первые для l_client_id 
new_leads = ldf[ldf['l_client_id']!='00000000-0000-0000-0000-000000000000'][['l_client_id']].merge(ldf, how='left', on='l_client_id')
new_leads = new_leads.assign(rn=new_leads.sort_values(["created_at"], ascending=True).groupby(["l_client_id"]).cumcount()+ 1).query('rn==1')

In [331]:
#ищем транзакции первые для l_client_id 
new_leads = new_leads.merge(tdf, how='left', on='l_client_id')
#новые заявки
new_leads = new_leads.assign(rn=new_leads.sort_values(["created_at_y"], ascending=True).groupby(["l_client_id"]).cumcount()+ 1).query('rn==1 and created_at_y >= created_at_x')


In [332]:
report['new_leads'] = new_leads.merge(mdf, how='left', right_on='manager_id', left_on='l_manager_id').groupby(['d_utm_source', 'd_club', 'd_manager'])['lead_id'].count()

In [319]:
#количество покупателей (кто купил в течение недели после заявки)
customers = ldf.merge(tdf, how='left', on='l_client_id')
customers =customers[(pd.to_datetime(customers['created_at_y'])-pd.to_datetime(customers['created_at_x']) <= pd.Timedelta('7 days')) & (pd.to_datetime(customers['created_at_y'])-pd.to_datetime(customers['created_at_x']) >= pd.Timedelta(seconds=0))]
report['customers'] = customers.merge(mdf, how='left', right_on='manager_id', left_on='l_manager_id').groupby(['d_utm_source', 'd_club', 'd_manager'])['l_client_id'].count()

In [322]:
#количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)
new_customers = ldf.merge(tdf, how='left', on='l_client_id')
new_customers = new_customers.assign(rn=new_customers.sort_values(["created_at_y"], ascending=True).groupby(["l_client_id"]).cumcount()+ 1).query('rn==1')
new_customers = new_customers[(pd.to_datetime(new_customers['created_at_y'])-pd.to_datetime(new_customers['created_at_x']) <= pd.Timedelta('7 days')) & (pd.to_datetime(new_customers['created_at_y'])-pd.to_datetime(new_customers['created_at_x']) >= pd.Timedelta(seconds=0))]
report['new_customers'] = new_customers.merge(mdf, how='left', right_on='manager_id', left_on='l_manager_id').groupby(['d_utm_source', 'd_club', 'd_manager'])['l_client_id'].count()

In [329]:
#доход от покупок новых покупателей
report['income_from_new_customers']=new_customers.merge(mdf, how='left', right_on='manager_id', left_on='l_manager_id').groupby(['d_utm_source', 'd_club', 'd_manager'])['m_real_amount'].sum()

In [326]:
from gspread_dataframe import set_with_dataframe

In [333]:
set_with_dataframe(sh.worksheet('report'), report.reset_index())